## Data loading (ISOT)

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split


real_df = pd.read_csv('real_clean.csv')
real = real_df['text'].values.tolist()
fake_df = pd.read_csv('Fake.csv')
fake = fake_df['text'].values.tolist()
real_examples = [str(x) for x in real]
fake_examples = [str(x) for x in fake]




print(f"Loading data...")
real_labels = [1] * len(real_examples)
fake_labels = [0] * len(fake_examples)

X = real_examples + fake_examples
y = real_labels + fake_labels

# Loop over the dataset to use tokenizer and truncate or zero pad
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42)

## BASELINE 1

In [ ]:
total_full_stop_train = []

for article in X_train:
    full_stop_count = 0
    for word in article:
        full_stop_count += word.count('.')
    total_full_stop_train.append(full_stop_count)

In [ ]:
dat_size = len(total_full_stop_train)
pos = sum(total_full_stop_train[:int(dat_size/2)]) / (dat_size/2)
neg = sum(total_full_stop_train[int(dat_size/2):]) / (dat_size/2)
print(pos)
print(neg)
boundary = (pos + neg) / 2
print(boundary)

In [ ]:
total_full_stop_val = []

for article in X_val:
    full_stop_count = 0
    for word in str(article):
        full_stop_count += word.count('.')
    total_full_stop_val.append(full_stop_count)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
predictions = [0 if x > boundary else 1 for x in total_full_stop_val]

acc = accuracy_score(y_val, predictions)
prec = precision_score(y_val, predictions)
rec = recall_score(y_val, predictions)
f1 = f1_score(y_val, predictions)

print('Accuracy: ', acc)
print('Precision: ', prec)
print('Recall: ', rec)
print('F1 Score: ', f1)

In [ ]:
total_full_stop_test = []

for article in X_test:
    full_stop_count = 0
    for word in str(article):
        full_stop_count += word.count('.')
    total_full_stop_test.append(full_stop_count)

In [ ]:
predictions = [0 if x > boundary else 1 for x in total_full_stop_test]

acc = accuracy_score(y_test, predictions)
prec = precision_score(y_test, predictions)
rec = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print('Accuracy: ', acc)
print('Precision: ', prec)
print('Recall: ', rec)
print('F1 Score: ', f1)

## BASELINE 2

In [ ]:
import gensim
import gensim.downloader

word2vec = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def score(model, word):
    try:
        return model[word]
    except KeyError:
        # Handle the case when one or both words are not in the vocabulary
        return np.zeros(300)

Train_Data = []

max_words = 10
print("Processing train articles ...")
# Iterate over each article
for article in X_train:
    # Create a matrix for the current article
    article_matrix = [score(word2vec, word) for word in article.split()[:10]]
    # Add the matrix to the list
    padding_size = max_words - len(article_matrix)
    if padding_size > 0:
        article_matrix += [np.zeros(300)] * padding_size
    Train_Data.append(np.asarray(article_matrix).flatten())


Train_Data = np.array(Train_Data)

print("Articles processed!")

model = LogisticRegression(max_iter=1000)

model.fit(Train_Data, y_train)

In [ ]:
Val_Data = []
print("Processing val articles ...")
# Iterate over each article
for article in X_val:
    # Create a matrix for the current article
    article_matrix = [score(word2vec, word) for word in article.split()[:10]]
    # Add the matrix to the list

    padding_size = max_words - len(article_matrix)
    if padding_size > 0:
        article_matrix += [np.zeros(300)] * padding_size
    Val_Data.append(np.asarray(article_matrix).flatten())

Val_Data = np.array(Val_Data)

print("Articles processed!")

predictions = model.predict(Val_Data)


acc = accuracy_score(y_val, predictions)
prec = precision_score(y_val, predictions)
rec = recall_score(y_val, predictions)
f1 = f1_score(y_val, predictions)


print('Scores for Baseline 2:')
print('Accuracy: ', acc)
print('Precision: ', prec)
print('Recall: ', rec)
print('F1 Score: ', f1)

In [ ]:
Test_Data = []
print("Processing test articles ...")
# Iterate over each article
for article in X_test:
    # Create a matrix for the current article
    article_matrix = [score(word2vec, word) for word in article.split()[:10]]
    # Add the matrix to the list
    padding_size = max_words - len(article_matrix)
    if padding_size > 0:
        article_matrix += [np.zeros(300)] * padding_size

    Test_Data.append(np.asarray(article_matrix).flatten())

Test_Data = np.array(Test_Data)

print("Articles processed!")

predictions = model.predict(Test_Data)


acc = accuracy_score(y_test, predictions)
prec = precision_score(y_test, predictions)
rec = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print('Scores for Baseline 2:')
print('Accuracy: ', acc)
print('Precision: ', prec)
print('Recall: ', rec)
print('F1 Score: ', f1)